In [5]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
# import nltk
import re
import joblib
import string

import requests

# url = ('https://newsapi.org/v2/top-headlines?'
#        'country=in&apiKey=53d38bf023184259914b8b77957eddf1')
# response = requests.get(url)
# articles = response.json()['articles']

# for article in articles:
#     print(article['title'], article['description'])

In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer

# cv = CountVectorizer()
# X = cv.fit_transform(text_data)  # where text_data is your list of news articles

In [6]:
fake = pd.read_csv('fake.csv')
true = pd.read_csv('true.csv')



In [8]:
fake.head()



,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [10]:

true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [11]:
fake['class']=0
true['class']=1

In [12]:
data=pd.concat([fake,true], axis=0)

In [13]:
data.sample(10)

,title,text,subject,date,class
15381,France says takes U.S. claims Iran violated U....,PARIS (Reuters) - France s foreign ministry sa...,worldnews,"November 8, 2017",1
8752,"In Virginia, Clinton tests Senator Tim Kaine a...","WASHINGTON/ANNANDALE, Virginia (Reuters) - Hil...",politicsNews,"July 14, 2016",1
17053,"Ardern to be next New Zealand PM, spelling cha...",WELLINGTON (Reuters) - New Zealand will get it...,worldnews,"October 18, 2017",1
10948,Trump says will attend March Fox debate with M...,(Reuters) - U.S. Republican presidential candi...,politicsNews,"February 5, 2016",1
6812,"Bernie Sanders Comforts Frustrated Voters, SH...","On Tuesday, Bernie Sanders slammed New York s ...",News,"April 19, 2016",0
17615,Mattis hopeful freeing of hostages in Pakistan...,MIAMI (Reuters) - U.S. Defense Secretary Jim M...,worldnews,"October 12, 2017",1
11931,AS RUSSIA STORY FALLS APART…Obama’s DOJ Goes A...,The inspector general will look at whether FBI...,politics,"Jan 12, 2017",0
11239,FOX PANELIST CLAIMS “Investigation” of Trump a...,"Mary Anne Marsh, a democrat, reveals that the ...",politics,"Apr 1, 2017",0
239,"Pence Joins Trump’s NFL Whinefest, Leaves Col...",People like to pretend that Mike Pence is the ...,News,"October 8, 2017",0
12226,FITTING END FOR COMMUNIST DICTATOR: Hilarious ...,As FoxNews.com reported:The Russian-made jeep ...,politics,"Dec 4, 2016",0


In [15]:
data=data.drop(["title","subject","date"], axis=1)

In [16]:
data.reset_index(inplace=True)

In [ ]:
# data.drop(['index'],axis=1,inplace=True)

In [23]:
data.sample(5)

,text,class
43411,"LAHORE, Pakistan (Reuters) - A Christian man h...",1
30339,(Reuters) - Retired Marine Corps General John ...,1
27225,(Reuters) - U.S. industry groups have submitte...,1
29637,WASHINGTON (Reuters) - The new U.S. administra...,1
18510,Former FBI Director James Comey admitted to th...,0


In [25]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'\\W', '', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>+', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    return text


In [26]:
data["text"]=data["text"].apply(clean_text)

In [29]:
x=data["text"]
y=data["class"]

xtrain, xtest, ytrain, ytest=train_test_split(x,y,test_size=0.25,random_state=42)

In [30]:
vectorizer=TfidfVectorizer()
xv_train=vectorizer.fit_transform(xtrain)
xv_test=vectorizer.transform(xtest)

In [31]:
lr=LogisticRegression()
lr.fit(xv_train,ytrain)


LogisticRegression()

In [32]:
prediction=lr.predict(xv_test)
lr.score(xv_test,ytest)

0.985924276169265

In [33]:
print(classification_report(ytest,prediction))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5895
           1       0.98      0.99      0.99      5330

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [34]:
joblib.dump(vectorizer,"vectorizer.jb")
joblib.dump(lr,"lr_model.jb")

['lr_model.jb']